### The purpose of this python notebook is to retrieve the hydraulic conductivity and retention capacity for the soils in the dataset using the staringreeks from Wageningen Environmental Research (WER).

Required inputs:
- Tabular dataset of soil sample points, with data for lutum fraction, soil organic carbon, and soil type classification for each point.
- Kriged raster map of groundwater depths in the area relative to sea level (NAP).
- A digital terrain model height map of the area. 

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.fill import fillnodata
from rasterio.plot import show

In [3]:
# Fill nodata in the heightmap
heightmap_path = './Data/ahn_ribu.tif'

with rio.open(heightmap_path) as src:
    profile = src.profile
    arr = src.read(1)
    arr_filled = fillnodata(arr, mask=src.read_masks(1), max_search_distance=50, smoothing_iterations=0)

with rio.open('./Data/heightmap_filled.tif', 'w', **profile) as dest:
    dest.write_band(1, arr_filled)

In [2]:
# Load the datasets
# Make sure to fill nodata value in the heightmap.

topsoil_data = gpd.read_file('./Data/topsoil_samples.gpkg')
subsoil_data = gpd.read_file('./Data/subsoil_samples.gpkg')
groundwater_raster = rio.open('./Data/kriged_groundwater.tif')
staringreeks_df = pd.read_csv('./Data/staringreeks_2018.csv')
staringreeks_2_df = pd.read_csv('./Data/unit_properties_2018.csv')
heightmap = rio.open('./Data/heightmap_filled.tif')

In [5]:
# For each point in the soil data, find the staringreeks entry that matches the soil type, then append the data to the soil data.
topsoil_data = topsoil_data.merge(
    staringreeks_df,
    left_on='type',
    right_on='name',
    how='left',
    suffixes=('', '_staringreeks')
)

In [5]:
subsoil_data = subsoil_data.merge(
    staringreeks_df,
    left_on='type',
    right_on='name',
    how='left',
    suffixes=('', '_staringreeks')
)

In [6]:
subsoil_data.head()

,datum,boorpunt_ID,BOVENKANT,ONDERKANT,analysemonster_ID,lutumgehalte,organisch_stofgehalte,leem,type,geometry,year,unit,name,wcr,wcs,alpha,npar,lambda,ksfit
0,NaT,8757,1.0,2.0,6009,2.0,NaN,NaN,None,POINT (121458 483788),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaT,12090,0.5,1.5,8617,0.1,0.6,21.6,O03,POINT (121647 483551),2018.0,21.0,O03,0.01,0.340,0.017,1.703,0.000,12.367
2,NaT,12091,0.5,1.5,8618,0.1,0.4,20.4,O03,POINT (121634 483498),2018.0,21.0,O03,0.01,0.340,0.017,1.703,0.000,12.367
3,NaT,13191,0.3,1.0,9537,12.0,1.6,27.6,O08,POINT (122053 484452),2018.0,26.0,O08,0.00,0.454,0.011,1.346,-0.904,8.641
4,NaT,13192,1.0,2.0,9538,7.0,1.3,25.8,O03,POINT (122053 484452),2018.0,21.0,O03,0.01,0.340,0.017,1.703,0.000,12.367


In [8]:
# Then, find the groundwater depth for each point in the topsoil data.
# Note that the groundwater raster is relative to sea level, so we need to subtract the heightmap from the groundwater raster.
# We can do this using a spatial join operation.

def calc_groundwater_depth (row):
    """Calculate groundwater depth for a given row."""
    if row['geometry'] is not None:
        # Get the groundwater value at the point
        groundwater_value = list(groundwater_raster.sample([(row.geometry.x, row.geometry.y)]))[0][0]
        if groundwater_value < 3e+38:
            h = groundwater_value * 100 # Convert to cm

            # Round h to the nearest of: 0, -10, -20, -31, -50, -100, -250, -500, -1000, -2500, -5000, -10000, -16000.
            if h >= 0:
                return 0
            elif -10 < h < 0:
                return -10
            elif -20 < h <= -10:
                return -20
            elif -31 < h <= -20:
                return -31
            elif -50 < h <= -31:
                return -50
            elif -100 < h <= -50:
                return -100
            elif -250 < h <= -100:
                return -250
            elif -500 < h <= -250:
                return -500
            elif h <= -500:
                return -1000
            
        else:
            return None
    else:
        return None


In [9]:
def get_wc(soil_unit, gw_height):
    wc_values = staringreeks_2_df[soil_unit]['wc']
    if gw_height in wc_values:
        return wc_values[gw_height]
    else:
        return None

In [11]:
def get_wc(unit, gw_height):
    try:
        unit_int = int(unit)
        gw_col = str(int(gw_height)) if float(gw_height).is_integer() else str(gw_height)
        row = staringreeks_2_df[(staringreeks_2_df['unit'] == unit_int) & (staringreeks_2_df['var'] == 'wc')]
        if not row.empty and gw_col in row.columns:
            return row.iloc[0][gw_col]
        else:
            return None
    except Exception:
        return None

def get_k(unit, gw_height):
    try:
        unit_int = int(unit)
        gw_col = str(int(gw_height)) if float(gw_height).is_integer() else str(gw_height)
        row = staringreeks_2_df[(staringreeks_2_df['unit'] == unit_int) & (staringreeks_2_df['var'] == 'k')]
        if not row.empty and gw_col in row.columns:
            return row.iloc[0][gw_col]
        else:
            return None
    except Exception:
        return None
    
def get_z1(unit, gw_height):
	try:
		unit_int = int(unit)
		gw_col = str(int(gw_height)) if float(gw_height).is_integer() else str(gw_height)
		row = staringreeks_2_df[(staringreeks_2_df['unit'] == unit_int) & (staringreeks_2_df['var'] == 'z1')]
		if not row.empty and gw_col in row.columns:
			return row.iloc[0][gw_col]
		else:
			return None
	except Exception:
		return None

def get_z2(unit, gw_height):
	try:
		unit_int = int(unit)
		gw_col = str(int(gw_height)) if float(gw_height).is_integer() else str(gw_height)
		row = staringreeks_2_df[(staringreeks_2_df['unit'] == unit_int) & (staringreeks_2_df['var'] == 'z2')]
		if not row.empty and gw_col in row.columns:
			return row.iloc[0][gw_col]
		else:
			return None
	except Exception:
		return None

In [ ]:
# Let's save the complete topsoil data with the groundwater depth and staringreeks data to a new file.
topsoil_data['groundwater_depth'] = topsoil_data.apply(calc_groundwater_depth, axis=1)
topsoil_data['wc'] = topsoil_data.apply(lambda row: get_wc(row['unit'], row['groundwater_depth']), axis=1)
topsoil_data['k'] = topsoil_data.apply(lambda row: get_k(row['unit'], row['groundwater_depth']), axis=1)
topsoil_data['z1'] = topsoil_data.apply(lambda row: get_z1(row['unit'], row['groundwater_depth']), axis=1)
topsoil_data['z2'] = topsoil_data.apply(lambda row: get_z2(row['unit'], row['groundwater_depth']), axis=1)
topsoil_data.to_file('./Data/topsoil_samples_complete.gpkg', driver='GPKG')

In [12]:
# Let's save the complete topsoil data with the groundwater depth and staringreeks data to a new file.
subsoil_data['groundwater_depth'] = subsoil_data.apply(calc_groundwater_depth, axis=1)
subsoil_data['wc'] = subsoil_data.apply(lambda row: get_wc(row['unit'], row['groundwater_depth']), axis=1)
subsoil_data['k'] = subsoil_data.apply(lambda row: get_k(row['unit'], row['groundwater_depth']), axis=1)
subsoil_data['z1'] = subsoil_data.apply(lambda row: get_z1(row['unit'], row['groundwater_depth']), axis=1)
subsoil_data['z2'] = subsoil_data.apply(lambda row: get_z2(row['unit'], row['groundwater_depth']), axis=1)
subsoil_data.to_file('./Data/subsoil_samples_complete.gpkg', driver='GPKG')